# 🐳 PARTIE 5 : DOCKER ET CONTAINERISATION

### 🌟 Ce que vous allez apprendre :
- **Containers** : Isolation et portabilité
- **Images** : Construction et gestion
- **Volumes** : Persistance des données
- **Networks** : Communication entre containers
- **Python** : Docker SDK pour l'automation

### 🛠️ Prérequis :
- Docker Desktop installé
- Exécutez d'abord la cellule système ci-dessous

---

In [3]:
# 📦 INSTALLATION DES PACKAGES NÉCESSAIRES
# Exécutez cette cellule en premier pour installer tous les packages requis

import subprocess
import sys

def install_package(package):
    """Installe un package Python via pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installé avec succès")
    except subprocess.CalledProcessError:
        print(f"❌ Erreur lors de l'installation de {package}")

# Liste des packages nécessaires pour ce notebook
packages = [
    "docker>=6.0.0",
    "ipywidgets>=7.6.0",
    "requests>=2.28.0",
    "pandas>=1.5.0"
]

print("🚀 Installation des packages pour Docker...")
print("Cela peut prendre quelques minutes...")

for package in packages:
    install_package(package)

print("\n✨ Installation terminée ! Vous pouvez maintenant exécuter les cellules suivantes.")
print("📝 Note: Redémarrez le kernel si nécessaire après l'installation.")

🚀 Installation des packages pour Docker...
Cela peut prendre quelques minutes...



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


✅ docker>=6.0.0 installé avec succès



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


✅ ipywidgets>=7.6.0 installé avec succès



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


✅ requests>=2.28.0 installé avec succès
✅ pandas>=1.5.0 installé avec succès

✨ Installation terminée ! Vous pouvez maintenant exécuter les cellules suivantes.
📝 Note: Redémarrez le kernel si nécessaire après l'installation.
✅ pandas>=1.5.0 installé avec succès

✨ Installation terminée ! Vous pouvez maintenant exécuter les cellules suivantes.
📝 Note: Redémarrez le kernel si nécessaire après l'installation.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# 🔧 SYSTÈME D'AIDE DOCKER (Exécuter une fois)
import sys
import os
from IPython.display import HTML, display
import ipywidgets as widgets
import json
from datetime import datetime
import requests
from pathlib import Path

# Tentative d'import Docker
try:
    import docker
    DOCKER_AVAILABLE = True
except ImportError:
    DOCKER_AVAILABLE = False

class DockerHelper:
    def __init__(self):
        self.success_style = """
        <div style="background: linear-gradient(90deg, #0db7ed, #0099cc); color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center; font-weight: bold; font-size: 16px;">
            🐳 {message} 🐳
        </div>
        """
        self.docker_dir = Path("docker_projects")
        self.docker_dir.mkdir(exist_ok=True)
        self.client = None
        
        # Base de données des aides cachées
        self.helps = {
            "5.1.1": {
                "hint": "Vérifiez que Docker Desktop est installé et démarré. Utilisez 'docker --version' en ligne de commande.",
                "solution": """# Vérification de Docker
import subprocess
import docker

# Test CLI Docker
try:
    result = subprocess.run(['docker', '--version'], capture_output=True, text=True)
    print(f"✅ Docker CLI: {result.stdout.strip()}")
except FileNotFoundError:
    print("❌ Docker CLI non trouvé")

# Test Docker SDK Python
try:
    client = docker.from_env()
    print(f"✅ Docker API: {client.version()['Version']}")
    client.close()
except Exception as e:
    print(f"❌ Docker API: {e}")""",
                "explanation": "Docker Desktop doit être installé et démarré pour que l'API Docker soit accessible depuis Python."
            },
            "5.1.2": {
                "hint": "Utilisez docker.from_env() pour créer un client Docker, puis client.containers.list() pour lister les conteneurs.",
                "solution": """import docker

# Connexion à Docker
client = docker.from_env()

# Lister tous les conteneurs (actifs et arrêtés)
all_containers = client.containers.list(all=True)
print(f"📦 Nombre total de conteneurs: {len(all_containers)}")

# Lister seulement les conteneurs actifs
running_containers = client.containers.list()
print(f"🟢 Conteneurs en cours d'exécution: {len(running_containers)}")

# Détails des conteneurs actifs
for container in running_containers:
    print(f"  - {container.name} ({container.image.tags[0] if container.image.tags else 'no-tag'})")

client.close()""",
                "explanation": "Le client Docker permet d'interagir avec l'API Docker pour gérer les conteneurs, images, volumes, etc."
            },
            "5.2.1": {
                "hint": "Utilisez client.images.pull() pour télécharger une image, puis client.containers.run() pour la démarrer.",
                "solution": """import docker

client = docker.from_env()

# Télécharger une image
print("📥 Téléchargement de l'image nginx...")
image = client.images.pull('nginx:alpine')
print(f"✅ Image téléchargée: {image.id[:12]}")

# Démarrer un conteneur
print("🚀 Démarrage du conteneur nginx...")
container = client.containers.run(
    'nginx:alpine',
    name='mon-nginx',
    ports={'80/tcp': 8080},
    detach=True
)

print(f"✅ Conteneur démarré: {container.name}")
print(f"🌐 Accessible sur: http://localhost:8080")

client.close()""",
                "explanation": "Docker permet de télécharger des images depuis Docker Hub et de les exécuter en tant que conteneurs."
            },
            "5.2.2": {
                "hint": "Utilisez container.stop() pour arrêter un conteneur, puis container.remove() pour le supprimer.",
                "solution": """import docker

client = docker.from_env()

try:
    # Récupérer le conteneur par son nom
    container = client.containers.get('mon-nginx')
    
    # Arrêter le conteneur
    print("🛑 Arrêt du conteneur...")
    container.stop()
    
    # Supprimer le conteneur
    print("🗑️ Suppression du conteneur...")
    container.remove()
    
    print("✅ Conteneur arrêté et supprimé")
    
except docker.errors.NotFound:
    print("❌ Conteneur 'mon-nginx' non trouvé")
except Exception as e:
    print(f"❌ Erreur: {e}")

client.close()""",
                "explanation": "La gestion du cycle de vie des conteneurs inclut leur arrêt et suppression pour libérer les ressources."
            },
            "5.3.1": {
                "hint": "Créez un Dockerfile avec FROM, COPY, RUN, EXPOSE et CMD. Utilisez client.images.build() pour construire.",
                "solution": """# Créer un Dockerfile simple
dockerfile_content = '''FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

EXPOSE 5000

CMD ["python", "app.py"]
'''

# Sauvegarder le Dockerfile
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

# Construire l'image
import docker
client = docker.from_env()

print("🔨 Construction de l'image...")
image, logs = client.images.build(
    path='.',
    tag='mon-app:latest',
    rm=True
)

print(f"✅ Image construite: {image.id[:12]}")
client.close()""",
                "explanation": "Un Dockerfile définit les étapes pour construire une image Docker personnalisée avec votre application."
            },
            "5.4.1": {
                "hint": "Utilisez client.volumes.create() pour créer un volume, puis montez-le avec volumes parameter dans run().",
                "solution": """import docker

client = docker.from_env()

# Créer un volume
volume = client.volumes.create(
    name='mon-volume',
    driver='local'
)
print(f"📁 Volume créé: {volume.name}")

# Utiliser le volume dans un conteneur
container = client.containers.run(
    'alpine',
    command='sh -c "echo Hello > /data/test.txt && cat /data/test.txt"',
    volumes={volume.name: {'bind': '/data', 'mode': 'rw'}},
    remove=True
)

print("✅ Données écrites dans le volume")

# Vérifier que les données persistent
container2 = client.containers.run(
    'alpine',
    command='cat /data/test.txt',
    volumes={volume.name: {'bind': '/data', 'mode': 'ro'}},
    remove=True
)

print("✅ Données lues depuis le volume persistant")
client.close()""",
                "explanation": "Les volumes Docker permettent de persister les données même après la suppression des conteneurs."
            }
        }
    
    def success_display(self, message):
        """Affiche un message de succès stylé"""
        display(HTML(self.success_style.format(message=message)))
    
    def connect_docker(self):
        """Établit une connexion Docker"""
        if not DOCKER_AVAILABLE:
            print("❌ Module docker non installé")
            print("📦 Installez avec: pip install docker")
            return False
        
        try:
            self.client = docker.from_env()
            version = self.client.version()
            print(f"✅ Connexion Docker réussie")
            print(f"🐳 Version Docker: {version['Version']}")
            return True
        except Exception as e:
            print(f"❌ Erreur de connexion Docker: {e}")
            print("💡 Vérifiez que Docker Desktop est démarré")
            return False
    
    def help(self, step_id=None):
        """Système d'aide intelligent pour Docker"""
        if step_id is None:
            print("🆘 AIDE DOCKER DISPONIBLE")
            print("="*40)
            print("📝 Usage: helper.help('5.X.Y')")
            print("\n🐳 Étapes disponibles:")
            for key in sorted(self.helps.keys()):
                print(f"  • {key}: {self.helps[key]['explanation'][:50]}...")
            return
        
        if step_id in self.helps:
            help_data = self.helps[step_id]
            
            # Interface interactive
            hint_btn = widgets.Button(description="💡 Indice", button_style='info')
            solution_btn = widgets.Button(description="✅ Solution", button_style='success')
            explain_btn = widgets.Button(description="📚 Explication", button_style='warning')
            
            output = widgets.Output()
            
            def show_hint(b):
                with output:
                    output.clear_output()
                    print(f"💡 INDICE pour {step_id}:")
                    print(help_data['hint'])
            
            def show_solution(b):
                with output:
                    output.clear_output()
                    print(f"✅ SOLUTION pour {step_id}:")
                    print(help_data['solution'])
            
            def show_explanation(b):
                with output:
                    output.clear_output()
                    print(f"📚 EXPLICATION pour {step_id}:")
                    print(help_data['explanation'])
            
            hint_btn.on_click(show_hint)
            solution_btn.on_click(show_solution)
            explain_btn.on_click(show_explanation)
            
            display(widgets.HBox([hint_btn, solution_btn, explain_btn]))
            display(output)
        else:
            print(f"❌ Aide non trouvée pour '{step_id}'")
            print("📝 Étapes disponibles:", list(self.helps.keys()))

# Initialisation du helper
helper = DockerHelper()
helper.success_display("Système d'aide Docker initialisé ! Utilisez helper.help() pour commencer")

print("\n🚀 DÉMARRAGE:")
print("1. Exécutez helper.connect_docker() pour tester Docker")
print("2. Utilisez helper.help('5.1.1') pour l'aide spécifique")
print("3. Les solutions complètes sont cachées dans l'aide interactive !")


🚀 DÉMARRAGE:
1. Exécutez helper.connect_docker() pour tester Docker
2. Utilisez helper.help('5.1.1') pour l'aide spécifique
3. Les solutions complètes sont cachées dans l'aide interactive !


---

## 📚 SECTION 5.1 : DÉCOUVERTE DE DOCKER

### 🎯 Objectifs :
- Comprendre les concepts de base de Docker
- Vérifier l'installation et la configuration
- Explorer l'API Docker depuis Python

In [ ]:
# 🐳 EXERCICE 5.1.1 : Vérification de Docker
# TODO: Vérifiez que Docker est installé et accessible
# AIDE: helper.help('5.1.1')

# Votre code ici

In [ ]:
# 🐳 EXERCICE 5.1.2 : Exploration des conteneurs
# TODO: Listez tous les conteneurs Docker (actifs et arrêtés)
# AIDE: helper.help('5.1.2')

# Votre code ici

---

## 📚 SECTION 5.2 : GESTION DES CONTENEURS

### 🎯 Objectifs :
- Démarrer et arrêter des conteneurs
- Gérer les ports et la configuration
- Comprendre le cycle de vie des conteneurs

In [ ]:
# 🐳 EXERCICE 5.2.1 : Démarrer un serveur web
# TODO: Téléchargez l'image nginx et démarrez un conteneur sur le port 8080
# AIDE: helper.help('5.2.1')

# Votre code ici

In [ ]:
# 🐳 EXERCICE 5.2.2 : Arrêter et nettoyer
# TODO: Arrêtez et supprimez le conteneur nginx précédent
# AIDE: helper.help('5.2.2')

# Votre code ici

---

## 📚 SECTION 5.3 : CONSTRUCTION D'IMAGES

### 🎯 Objectifs :
- Créer un Dockerfile
- Construire des images personnalisées
- Comprendre les layers et le cache

In [ ]:
# 🐳 EXERCICE 5.3.1 : Créer une image personnalisée
# TODO: Créez un Dockerfile et construisez une image Python simple
# AIDE: helper.help('5.3.1')

# Votre code ici

---

## 📚 SECTION 5.4 : VOLUMES ET PERSISTANCE

### 🎯 Objectifs :
- Créer et gérer des volumes
- Comprendre la persistance des données
- Monter des volumes dans les conteneurs

In [ ]:
# 🐳 EXERCICE 5.4.1 : Volumes persistants
# TODO: Créez un volume et testez la persistance des données
# AIDE: helper.help('5.4.1')

# Votre code ici

---

## 🎊 FÉLICITATIONS !

Vous avez terminé le module Docker ! Vous maîtrisez maintenant :

### ✅ Ce que vous avez appris :
- **Conteneurs** : Isolation et exécution d'applications
- **Images** : Construction et gestion d'environnements
- **Volumes** : Persistance et partage de données
- **API Python** : Automation Docker avec le SDK

### 🚀 Prochaines étapes :
- Explorez Docker Compose pour les applications multi-conteneurs
- Découvrez les registries Docker (Docker Hub, privés)
- Apprenez l'orchestration avec Kubernetes

**🎯 Continuez vers la Partie 6 : MongoDB et NoSQL !**